# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [71]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [63]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [64]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [65]:
data.shape

(68584, 7)

In [66]:
data["CustomerID"].value_counts

<bound method IndexOpsMixin.value_counts of 0        61288
1        77352
2        40094
3        23548
4        78981
         ...  
68579    94547
68580    57654
68581    36751
68582    43270
68583    78718
Name: CustomerID, Length: 68584, dtype: int64>

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [67]:
df=data.groupby(['CustomerID','ProductName']).agg({"Quantity":'sum'})
df=df.reset_index()


df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [68]:
pivot = pd.pivot_table(df, values="Quantity", index=['ProductName'], columns=['CustomerID'], fill_value = 0)

pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [175]:
distances = pdist(pivot, 'correlation')
print(distances)
print("-----Squareform-----")
print(squareform(distances))

[1.00351885 1.00189923 0.88484387 ... 1.00041902 0.9538408  0.93220909]
-----Squareform-----
[[0.         1.00351885 1.00189923 ... 0.97621369 0.93035757 1.00408193]
 [1.00351885 0.         1.00065452 ... 0.9251123  0.9666788  0.95045238]
 [1.00189923 1.00065452 0.         ... 0.96267389 0.95337254 0.99762974]
 ...
 [0.97621369 0.9251123  0.96267389 ... 0.         1.00041902 0.9538408 ]
 [0.93035757 0.9666788  0.95337254 ... 1.00041902 0.         0.93220909]
 [1.00408193 0.95045238 0.99762974 ... 0.9538408  0.93220909 0.        ]]


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [180]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'correlation'))), #changed for step 10
                         index=pivot.columns, columns=pivot.columns)
distances.reset_index().columns

Index(['CustomerID',           33,          200,          264,          356,
                412,          464,          477,          639,          649,
       ...
              97697,        97753,        97769,        97793,        97900,
              97928,        98069,        98159,        98185,        98200],
      dtype='object', name='CustomerID', length=1001)

In [181]:
def findSimilar(custID):
    return [n for n in distances[custID].sort_values(ascending=False)[1:].head().index]

## I will be using the customerID 98200 from now on
findSimilar(98200)

[58639, 14782, 66709, 78488, 11883]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [178]:
df_step5 = data.loc[data['CustomerID'].isin(findSimilar(98200))]


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [144]:
df_step6 = df_step5.groupby(['ProductName']).agg({"Quantity":'sum'}).sort_values(by='Quantity',ascending=False).reset_index()
df_step6

,ProductName,Quantity
0,Sea Bass - Whole,30
1,Beans - Wax,27
2,Apricots - Halves,27
3,Sherry - Dry,26
4,Pork - Hock And Feet Attached,22
...,...,...
248,General Purpose Trigger,4
249,Guinea Fowl,4
250,Halibut - Steaks,4
251,"Juice - Cranberry, 341 Ml",4


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [160]:
list_all = [df_step6["ProductName"][e] for e in range(df_step6.shape[0])]

data_step7 = data.loc[data['CustomerID'] == 98200]
list_customer = [e for e in data_step7["ProductName"]]

for e in list_customer:
    if e in list_all:
        list_all.remove(e)


print("Here are the 5 elements you should buy:")
for e in list_all[:5]:
    print(f"-> {e}")

Here are the 5 elements you should buy:
-> Beans - Wax
-> Sherry - Dry
-> Pork - Hock And Feet Attached
-> Pears - Bosc
-> Soup - Campbells, Cream Of


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [167]:
recommendations=dict()
CustomersIDS=list(set(data['CustomerID']))
len(CustomersIDS)


for user in CustomersIDS:
    top5 = findSimilar(user)
    df = data.loc[data['CustomerID'].isin(findSimilar(user))]
    df_2 = df.groupby(['ProductName']).agg({"Quantity":'sum'}).sort_values(by='Quantity',ascending=False).reset_index()
    list_all = [df_2["ProductName"][e] for e in range(df_2.shape[0])]
    data_step7 = data.loc[data['CustomerID'] == user]
    list_customer = [e for e in data_step7["ProductName"]]
    
    for e in list_customer:
        if e in list_all:
            list_all.remove(e)
            
    recommendations[user] = dict(zip(list(range(1,6)), list_all[:5])) 
    

                                


In [172]:
len(recommendations)

1000

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [174]:
pd.DataFrame(recommendations).T

,1,2,3,4,5
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
...,...,...,...,...,...
77814,Cheese - Victor Et Berthold,Cake - Cake Sheet Macaroon,Sea Bass - Whole,Squid U5 - Thailand,Guinea Fowl
32753,Grouper - Fresh,Sherry - Dry,"Chestnuts - Whole,canned",Napkin White - Starched,Sprouts - Alfalfa
20476,Soup - Campbells Tomato Ravioli,"Pasta - Detalini, White, Fresh","Cheese - Boursin, Garlic / Herbs",Beans - Kidney White,Wine - Blue Nun Qualitatswein
67582,Cake - Mini Cheesecake,Ecolab - Solid Fusion,Wine - Redchard Merritt,Mussels - Frozen,Cheese - Bocconcini


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [182]:
recommendations=dict()
CustomersIDS=list(set(data['CustomerID']))
len(CustomersIDS)


for user in CustomersIDS:
    top5 = findSimilar(user)
    df = data.loc[data['CustomerID'].isin(findSimilar(user))]
    df_2 = df.groupby(['ProductName']).agg({"Quantity":'sum'}).sort_values(by='Quantity',ascending=False).reset_index()
    list_all = [df_2["ProductName"][e] for e in range(df_2.shape[0])]
    data_step7 = data.loc[data['CustomerID'] == user]
    list_customer = [e for e in data_step7["ProductName"]]
    
    for e in list_customer:
        if e in list_all:
            list_all.remove(e)
            
    recommendations[user] = dict(zip(list(range(1,6)), list_all[:5])) 
    

In [183]:
pd.DataFrame(recommendations).T

,1,2,3,4,5
83973,Pastry - Choclate Baked,Flour - Pastry,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn
59399,Anchovy Paste - 56 G Tube,Ezy Change Mophandle,Garlic - Peeled,Beans - Wax,Broom - Corn
92168,Wiberg Super Cure,Tuna - Salad Premix,Sage - Ground,Apricots Fresh,Beef - Inside Round
49159,"Turnip - White, Organic",Peas - Frozen,Lamb - Ground,Beer - Blue,Beef - Ground Medium
18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Ecolab - Lime - A - Way 4/4 L,Milk - 2%
...,...,...,...,...,...
77814,Hersey Shakes,Potatoes - Idaho 100 Count,Beef - Texas Style Burger,Scallops - Live In Shell,Curry Paste - Madras
32753,Rice - Jasmine Sented,Tuna - Salad Premix,Beef - Ground Medium,"Water, Tap",Sun - Dried Tomatoes
20476,"Beer - Alexander Kieths, Pale Ale",Grapes - Red,Lettuce - Frisee,Table Cloth 62x114 White,Onion Powder
67582,"Oil - Shortening,liqud, Fry",Tea - Herbal Sweet Dreams,Crab - Imitation Flakes,Cumin - Whole,Flour - Pastry
